In [2]:
# Import Dependencies
import pandas as pd
import numpy as np
from census import Census
from config import census_key
import cpi # this will be used to determine the cost of everything in today's dollars. cpi.inflate(amount,year of amount)
#install cpi with pip install cpi on command line
cpi.update()
low_memory=False

In [3]:
# Import CSVs 
filepath = "Resources/"
kent_zipcodes = pd.read_csv(filepath+"kent_zipcodes.csv",header=None)
oakland_zipcodes = pd.read_csv(filepath+"oakland_zipcodes.csv",header=None)
saginaw_zipcodes = pd.read_csv(filepath+"saginaw_zipcodes.csv",header=None)
muskegon_zipcodes = pd.read_csv(filepath+"muskegon_zipcodes.csv",header=None)

In [4]:
# Create dataframe for each imported CSV and add a column with the corresponding county
kent_zipcodes = pd.DataFrame(kent_zipcodes)
kent_zipcodes["County"] = "Kent"
oakland_zipcodes = pd.DataFrame(oakland_zipcodes)
oakland_zipcodes["County"] = "Oakland"
saginaw_zipcodes = pd.DataFrame(saginaw_zipcodes)
saginaw_zipcodes["County"] = "Saginaw"
muskegon_zipcodes = pd.DataFrame(muskegon_zipcodes)
muskegon_zipcodes["County"] = "Muskegon"
# Merge zipcode dataframes and add a column name for Zipcode
query_zipcodes = pd.DataFrame(pd.merge(kent_zipcodes,oakland_zipcodes,how="outer"))
query_zipcodes = pd.DataFrame(pd.merge(query_zipcodes,saginaw_zipcodes,how="outer"))
query_zipcodes = pd.DataFrame(pd.merge(query_zipcodes,muskegon_zipcodes,how="outer"))
query_zipcodes.columns = ["Zipcode","County"]
query_zipcodes = query_zipcodes.astype({"Zipcode":object})

In [5]:
query_zipcodes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Zipcode  188 non-null    object
 1   County   188 non-null    object
dtypes: object(2)
memory usage: 3.1+ KB


In [6]:
# Create dataframe to add census data to. Needed to put in starter values for it to function properly later. Will remove further down.
census_df = pd.DataFrame({"Year":[0],
                          "Zipcode":["0"],
                          "Total Population":[0],
                          "White Population":[0],
                          "Black Population":[0],
                          "Employed Count":[0],
                          "Unemployed Count":[0],
                          "Poverty Count":[0],
                          "Poverty Count - White":[0],
                          "Poverty Count - Black":[0],
                          "Median Income":[0],
                          "Median Rent":[0],
                          "Median Home Value":[0]})

# Get data from API for Census
census_df = census_df.astype({"Year":int,"Zipcode":object})

# For loop to iterate through the desired 10-year period
for year in [2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]:
    
    c = Census(census_key,year) 
    census_data = c.acs5.get(("NAME", "B19013_001E", "B23025_004E", "B23025_005E", "B17001_002E", 
            "B17001A_002E","B17001B_002E", "B01003_001E",
            "B02001_002E", "B02001_003E","B25058_001E", "B25077_001E"
             ), {"for": "zip code tabulation area:*"})
    census_raw_df = pd.DataFrame(census_data)
    
# Add the year to the year column so each iteration can be distinguished
    year_count = len(census_data)
    year_series = pd.Series([year for x in range(year_count)])    
    census_raw_df["Year"] = year_series
    
# Rename Columns
    census_raw_df = census_raw_df.rename(columns = {
            "B19013_001E" : "Median Income", 
            "B23025_004E" : "Employed Count", 
            "B23025_005E" : "Unemployed Count", 
            "B17001_002E" : "Poverty Count", 
            "B17001A_002E" : "Poverty Count - White" , 
            "B17001B_002E" : "Poverty Count - Black" , 
            "B01003_001E" : "Total Population", 
            "B02001_002E" : "White Population", 
            "B02001_003E" : "Black Population", 
            "B25058_001E" : "Median Rent", 
            "B25077_001E" : "Median Home Value", 
            "NAME" : "Name",
            "zip code tabulation area": "Zipcode"})
    
# Recast all variables in the output dataframe to the correct type 
    census_raw_df = census_raw_df.astype({"Year":int,"Zipcode":object,"Median Income":float,"Employed Count":float,"Unemployed Count":float,"Poverty Count":float,
             "Poverty Count - White":float, "Poverty Count - Black":float, "Total Population":float,
             "White Population":float,"Black Population":float, "Median Rent":float,"Median Home Value":float})
    
# Recast the Zipcode in census_df to object. Was running into issues with the zipcode repeatedly trying to cast as a float.
    census_df = census_df.astype({"Zipcode":object})
    
# Merge the new raw df into the finished df. Use outer to maintain all values
    census_df = pd.merge(census_df,census_raw_df,how="outer")
    

In [7]:
for i in query_zipcodes.index:
    query_zipcodes.at[i,"Zipcode"] = str(query_zipcodes.loc[i,"Zipcode"])
query_zipcodes

,Zipcode,County
0,49508,Kent
1,49504,Kent
2,49507,Kent
3,49503,Kent
4,49341,Kent
...,...,...
183,49440,Muskegon
184,49318,Muskegon
185,49303,Muskegon
186,49443,Muskegon


In [8]:
# Merge Dataframes so only those zipcodes we want are there. Also adds County 
census_df = pd.merge(census_df,query_zipcodes,how="inner", on="Zipcode")

In [9]:
# Remove the name column, which is useless here, and the state column, which only functioned for some years
census_df = census_df.drop(columns = ["Name","state"])

#Remove the sample data added to initialize the dataframe
census_df = census_df.loc[census_df["Year"] != 0]

In [10]:
census_df.sort_values("Median Income")

,Year,Zipcode,Total Population,White Population,Black Population,Employed Count,Unemployed Count,Poverty Count,Poverty Count - White,Poverty Count - Black,Median Income,Median Rent,Median Home Value,County
886,2017,48724,458.0,398.0,60.0,217.0,0.0,136.0,136.0,0.0,-666666666.0,-666666666.0,70900.0,Saginaw
887,2018,48724,484.0,423.0,61.0,227.0,0.0,145.0,145.0,0.0,-666666666.0,-666666666.0,71900.0,Saginaw
888,2019,48724,544.0,470.0,74.0,241.0,20.0,227.0,227.0,0.0,-666666666.0,-666666666.0,70000.0,Saginaw
1165,2016,49440,807.0,580.0,89.0,243.0,77.0,326.0,200.0,57.0,14634.0,476.0,88300.0,Muskegon
1167,2018,49440,983.0,734.0,106.0,337.0,79.0,399.0,244.0,74.0,14649.0,468.0,113600.0,Muskegon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,2018,48306,27663.0,23587.0,858.0,13366.0,412.0,469.0,411.0,4.0,147303.0,853.0,386500.0,Oakland
564,2015,48374,15316.0,10717.0,594.0,7809.0,324.0,999.0,571.0,87.0,147536.0,1640.0,367800.0,Oakland
239,2020,48301,15024.0,12160.0,841.0,7182.0,357.0,421.0,365.0,20.0,147585.0,1443.0,544500.0,Oakland
89,2020,48070,6303.0,5869.0,62.0,3500.0,65.0,92.0,92.0,0.0,150114.0,1754.0,372200.0,Oakland


In [11]:
census_df = census_df.loc[(census_df["Zipcode"]!= "48724") & (census_df["Zipcode"]!= "48370") & (census_df["Zipcode"]!= "48380")]

In [12]:
census_df.sort_values("Median Rent").head(20)

,Year,Zipcode,Total Population,White Population,Black Population,Employed Count,Unemployed Count,Poverty Count,Poverty Count - White,Poverty Count - Black,Median Income,Median Rent,Median Home Value,County
671,2012,48417,2830.0,2785.0,0.0,1187.0,209.0,426.0,399.0,0.0,42361.0,290.0,106000.0,Saginaw
911,2012,48807,800.0,777.0,0.0,339.0,51.0,70.0,67.0,0.0,49135.0,300.0,116800.0,Saginaw
672,2013,48417,2741.0,2699.0,0.0,1075.0,180.0,311.0,285.0,0.0,46382.0,308.0,103000.0,Saginaw
764,2015,48607,1305.0,215.0,968.0,452.0,54.0,542.0,113.0,385.0,20994.0,338.0,38800.0,Saginaw
762,2013,48607,1392.0,294.0,949.0,445.0,119.0,604.0,179.0,409.0,21302.0,368.0,45200.0,Saginaw
760,2011,48607,1813.0,355.0,1373.0,361.0,169.0,873.0,212.0,648.0,15552.0,376.0,62200.0,Saginaw
765,2016,48607,1236.0,245.0,815.0,357.0,49.0,540.0,105.0,378.0,20710.0,378.0,40700.0,Saginaw
782,2013,48614,1375.0,1357.0,1.0,517.0,65.0,310.0,309.0,1.0,48942.0,378.0,104300.0,Saginaw
763,2014,48607,1584.0,286.0,1165.0,522.0,132.0,821.0,163.0,622.0,20833.0,382.0,43900.0,Saginaw
761,2012,48607,1628.0,286.0,1255.0,365.0,165.0,761.0,161.0,589.0,17695.0,390.0,63100.0,Saginaw


In [14]:
# Adjust financial columns to account for inflation. All values go to current-day inflation (2023)
for i in census_df.index:
    census_df.at[i,"Adjusted Median Income"] = cpi.inflate((census_df.at[i,"Median Income"]),(census_df.at[i,"Year"]))
    census_df.at[i,"Adjusted Median Rent"] = cpi.inflate((census_df.at[i,"Median Rent"]),(census_df.at[i,"Year"]))
    census_df.at[i,"Adjusted Median Home Value"] = cpi.inflate((census_df.at[i,"Median Home Value"]),(census_df.at[i,"Year"]))

In [15]:
census_df.to_csv("datasets/census.csv",index_label="datapoint")